In [1]:
# 对汽车图片数据进行增强
import os
import matplotlib.pyplot as plt
import cv2

# 数据格式 
# Vehicle/images/train/vehicle_0000002.jpg  jpg图片格式
# Vehicle/labels/train/vehicle_0000002.txt  
# txt内容 0 0.5 0.5 0.5 0.5  第一个数是标签，
# 后四个数是car在图片中的矩形框框左上和右下点x1, y1, x2, y2坐标相对于图片的比例
train_path = '/home/ubuntu/文档/GitHub/DL大作业/datasets/Vehicle_enhance/images/train'
val_path = '/home/ubuntu/文档/GitHub/DL大作业/datasets/Vehicle_enhance/images/val'

train_labels_path = '/home/ubuntu/文档/GitHub/DL大作业/datasets/Vehicle_enhance/labels/train'
val_labels_path = '/home/ubuntu/文档/GitHub/DL大作业/datasets/Vehicle_enhance/labels/val'


# 展示一下数据集, 展示图片和对应label文件中框框的位置
def get_label(image_name):
    image_labels_path = os.path.join(train_labels_path, image_name + '.txt')
    with open(image_labels_path, 'r') as f:
        label = f.readline()
        label = label.split(' ')
        label = [float(x) for x in label]
        return label


def show_image(image_name):
    image_path = os.path.join(train_path, image_name + '.jpg')
    # 使用plt展示图片
    image = plt.imread(image_path)
    plt.imshow(image)
    # 读取label文件
    class_id, x_center, y_center, width, height = get_label(image_name)
    x1 = int((x_center - (width / 2)) * image.shape[1])
    y1 = int((y_center - (height / 2)) * image.shape[0])
    x2 = int((x_center + (width / 2)) * image.shape[1])
    y2 = int((y_center + (height / 2)) * image.shape[0])
    plt.gca().add_patch(plt.Rectangle((x1, y1), x2 - x1, y2 - y1, fill=False, edgecolor='red', linewidth=2))

    print(x1, y1, x2, y2)
    print(image.shape[0], ' ', image.shape[1])

# 随机颜色,对比,亮度变化， 变化幅度不要太大
def random_color(image):
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    # 随机颜色
    color_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    color_image[:, :, 0] = color_image[:, :, 0] * (1 + random.uniform(-0.5, 0.5))
    color_image[:, :, 1] = color_image[:, :, 1] * (1 + random.uniform(-0.5, 0.5))
    color_image[:, :, 2] = color_image[:, :, 2] * (1 + random.uniform(-0.5, 0.5))
    color_image = cv2.cvtColor(color_image, cv2.COLOR_HSV2BGR)
    return color_image

# 运用上述三种方法加上原图，对数据集进行增强

enhanced_train_path = '/home/ubuntu/文档/GitHub/DL大作业/datasets/VehicleEnhanced/images/train'
enhanced_val_path = '/home/ubuntu/文档/GitHub/DL大作业/datasets/VehicleEnhanced/images/val'
enhanced_train_labels_path = '/home/ubuntu/文档/GitHub/DL大作业/datasets/VehicleEnhanced/labels/train'
enhanced_val_labels_path = '/home/ubuntu/文档/GitHub/DL大作业/datasets/VehicleEnhanced/labels/val'

# 清空上述四个文件夹
import shutil

shutil.rmtree(enhanced_train_path, ignore_errors=True)
shutil.rmtree(enhanced_val_path, ignore_errors=True)
shutil.rmtree(enhanced_train_labels_path, ignore_errors=True)
shutil.rmtree(enhanced_val_labels_path, ignore_errors=True)

# 若原图名为 vehicle_0000002.jpg, 
# 则增强后的图像名为 
# vehicle_0000002_0.jpg,   原图
# vehicle_0000002_1.jpg,  翻转图
# vehicle_0000002_2.jpg,  遮盖图
# vehicle_0000002_3.jpg   随机颜色，对比度，亮度变化图
# label文件名也要对应增加，翻转图的label文件中x_center要变为1-x_center
# 其余label文件除了文件名均保持不变

import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import random


def get_label(image_name, labels_path):
    image_labels_path = os.path.join(labels_path, image_name + '.txt')
    with open(image_labels_path, 'r') as f:
        label = f.readline().strip().split()
        label = [float(x) for x in label]
        return label


def save_image_and_label(image, label, new_image_name, new_image_path, new_labels_path):
    # label[0] 是类别标签，应当为整数
    label_str = str(int(label[0]))  # 类别标签转换为整数，防止保存为小数
    # label[1:] 是坐标值，这些保持浮点数格式
    label_str += ' ' + ' '.join(map(str, label[1:]))  # 其他标签保持原格式
    plt.imsave(os.path.join(new_image_path, new_image_name + '.jpg'), image)
    with open(os.path.join(new_labels_path, new_image_name + '.txt'), 'w') as f:
        f.write(label_str + '\n')


def process_dataset(image_path, labels_path, enhanced_image_path, enhanced_labels_path):
    # 创建增强目录
    os.makedirs(enhanced_image_path, exist_ok=True)
    os.makedirs(enhanced_labels_path, exist_ok=True)
    print("开始处理数据集：", image_path)
    print(len(os.listdir(image_path)))
    for filename in os.listdir(image_path):

        if filename.endswith('.jpg'):
            print("正在处理：", filename)
            image_name = filename[:-4]
            original_image_path = os.path.join(image_path, filename)
            image = plt.imread(original_image_path)

            # 原始图像
            label = get_label(image_name, labels_path)
            save_image_and_label(image, label, image_name + '_0', enhanced_image_path, enhanced_labels_path)

            # 翻转图像
            flip_image = cv2.flip(image, 1)
            flip_label = label.copy()
            flip_label[1] = 1 - flip_label[1]  # 更新x_center
            save_image_and_label(flip_image, flip_label, image_name + '_1', enhanced_image_path, enhanced_labels_path)

            # 遮盖图像
            cover_image = image.copy()
            x1 = int((label[1] - (label[3] / 2)) * cover_image.shape[1])
            y1 = int((label[2] - (label[4] / 2)) * cover_image.shape[0])
            x2 = int((label[1] + (label[3] / 2)) * cover_image.shape[1])
            y2 = int((label[2] + (label[4] / 2)) * cover_image.shape[0])
            mask_x = (x2 - x1) // 3
            mask_y = (y2 - y1) // 3
            mask_x1 = random.randint(x1, x2 - mask_x)
            mask_y1 = random.randint(y1, y2 - mask_y)
            mask_x2 = mask_x1 + mask_x
            mask_y2 = mask_y1 + mask_y
            cover_image[mask_y1:mask_y2, mask_x1:mask_x2] = 255
            save_image_and_label(cover_image, label, image_name + '_2', enhanced_image_path, enhanced_labels_path)

            # 随机颜色，对比度，亮度变化图
            color_image = random_color(image)
            save_image_and_label(color_image, label, image_name + '_3', enhanced_image_path, enhanced_labels_path)

# Process training and validation datasets
process_dataset(train_path, train_labels_path, enhanced_train_path, enhanced_train_labels_path)
process_dataset(val_path, val_labels_path, enhanced_val_path, enhanced_val_labels_path)

开始处理数据集： /home/ubuntu/文档/GitHub/DL大作业/datasets/Vehicle_enhance/images/train
8865
正在处理： vehicle_0001722.jpg
正在处理： vehicle_0008904.jpg
正在处理： vehicle_0006429.jpg
正在处理： vehicle_0003085.jpg
正在处理： vehicle_0003603.jpg
正在处理： vehicle_0000765.jpg
正在处理： vehicle_0004709.jpg
正在处理： vehicle_0006280.jpg
正在处理： vehicle_0004427.jpg
正在处理： vehicle_0003738.jpg
正在处理： vehicle_0001443.jpg
正在处理： vehicle_0001363.jpg
正在处理： vehicle_0007183.jpg
正在处理： vehicle_0006689.jpg
正在处理： vehicle_0004854.jpg
正在处理： vehicle_0008524.jpg
正在处理： vehicle_0002638.jpg
正在处理： vehicle_0002220.jpg
正在处理： vehicle_0004514.jpg
正在处理： vehicle_0009788.jpg
正在处理： vehicle_0005114.jpg
正在处理： vehicle_0007172.jpg
正在处理： vehicle_0009813.jpg
正在处理： vehicle_0006069.jpg
正在处理： vehicle_0001796.jpg
正在处理： vehicle_0003545.jpg
正在处理： vehicle_0007986.jpg
正在处理： vehicle_0007867.jpg
正在处理： vehicle_0001756.jpg
正在处理： vehicle_0000854.jpg
正在处理： vehicle_0006123.jpg
正在处理： vehicle_0007716.jpg
正在处理： vehicle_0000477.jpg
正在处理： vehicle_0006324.jpg
正在处理： vehicle_0008232.jpg
正在处理： veh